In [2]:
import numpy as np
import scipy as sc
# la funcionalidad específica de robótica está en el toolbox
import roboticstoolbox as rtb
# spatialmath tiene define los grupos de transformaciones especiales: rotación SO3 y eculideo SE3
import spatialmath as sm
import spatialmath.base.symbolic as sym
import matplotlib.pyplot as plt  

In [7]:
# Esto fue un papelón. Ahora voy a sumarle el calculo del PCI analítico
class irb140_clase(rtb.DHRobot):
    def __init__(self, *args, **kwargs):
        super().__init__([
        rtb.RevoluteDH(alpha=-np.pi/2,a=0.07),
        rtb.RevoluteDH(a=0.36,offset=-np.pi/2),
        #rtb.RevoluteDH(a=0.36),
        rtb.RevoluteDH(alpha=np.pi/2,offset=np.pi),
        #rtb.RevoluteDH(alpha=np.pi/2),
        rtb.RevoluteDH(d=0.38, alpha=-np.pi/2),
        rtb.RevoluteDH(alpha=np.pi/2),
        rtb.RevoluteDH()
        ], name="IRB140")

    def get_config(self,q):
        g1 = np.sign(self.links[3].d * np.sin(q[1]+self.links[1].offset +q[2]+self.links[2].offset) + self.links[1].a * np.cos(q[1]+self.links[1].offset) + self.links[0].a)
        g2 = np.sign(np.cos(q[2]+self.links[2].offset))
        g3 = np.sign(np.sin(q[4]+self.links[4].offset))
        return np.array([g1,g2,g3])
    
    def ikine_a(self,POSE,conf,q1_actual=0,q4_actual=0):
        # Problema inversor para el robot ABB IRB 140
        # Se determina el vector q

        status = -1

        # Inicializo el vector de variables articulares
        q = np.zeros(6); 

        # Resuelvo el problema de posicion    
        px = POSE.t[0]
        py = POSE.t[1]
        pz = POSE.t[2]

        # Calculo q1 
        s1 = conf[0]*py
        c1 = conf[0]*px
        if px==0 and py==0:
            q[0] = q1_actual
        else:
            q[0] = np.arctan2(s1,c1)

        # Calculo q3 (codo)
        s3 = ((px*np.cos(q[0]) + py*np.sin(q[0]) - self.links[0].a)**2 + pz**2 - self.links[1].a**2 - self.links[3].d**2)/(2*self.links[1].a*self.links[3].d);
        if (np.abs(s3) > 1):
            print('ERROR: Punto no alcanzable.')
            return q,status
        c3 = conf[1]*np.sqrt(1-s3**2)
        q[2] = np.arctan2(s3,c3)        

        # Calculo q2
        s2 = (px*np.cos(q[0]) + py*np.sin(q[0]) - self.links[0].a)*(self.links[3].d*np.cos(q[2])) - (self.links[3].d*np.sin(q[2]) + self.links[1].a)*pz
        c2 = (px*np.cos(q[0]) + py*np.sin(q[0]) - self.links[0].a)*(self.links[3].d*np.sin(q[2]) + self.links[1].a) + (self.links[3].d*np.cos(q[2]))*pz
        q[1] = np.arctan2(s2,c2)

        # Calculo el problema de orientacion
        # Calculo R_63
        A60 = self.fkine(q-self.offset)
        # Hasta el momento la parte de rotación es R30
        R_30 = A60.R

        R_63 = R_30.T @ POSE.R
        # Calculo q(5)
        c5 = R_63[2,2]
        q[4] = np.arctan2(conf[2]*np.sqrt(1-c5**2),c5)

        if np.abs(R_63[1,2])<1E-9 and np.abs(R_63[0,2])<1E-9:
            q[3] = q4_actual
            q[5] = np.arctan2(R_63[1,0],R_63[0,0])-q4_actual
        else:
            # Calculo de q(4)(1-c
            s4 = conf[2]*R_63[1,2]
            c4 = conf[2]*R_63[0,2]
            q[3] = np.arctan2(s4,c4)

            # Calculo de q(6)
            s6 = conf[2]*R_63[2,1]
            c6 = -conf[2]*R_63[2,0]
            q[5] = np.arctan2(s6,c6)

        status = 0

        # Sumo el offset y limito entre -pi y pi
        q = np.angle(np.exp(1j*(q-self.offset)))
        return q,status

In [31]:
def ik_iterative(robot, Tdes, q0, conf=None, tol=1e-3, max_iter=1000, lam=0.01, Ts=0.05, v_d=1.0, alpha=0.01, verbose=False):
    """
    Cinemática inversa numérica iterativa basada en Jacobiana con orientación y configuración deseada.
    Parámetros:
        robot   : objeto DH de Robotics Toolbox (ej. irb140_clase())
        Tdes    : Pose deseada (SE3 o matriz homogénea 4x4)
        q0      : Configuración inicial (vector articular)
        conf    : vector de configuración deseada (ej. [±1, ±1, ±1]) o None
        tol     : tolerancia de error en norma de pose
        max_iter: máximo de iteraciones permitidas
        lam     : damping factor para evitar singularidades
        Ts      : paso de integración
        v_d     : velocidad deseada en espacio cartesiano
        alpha   : peso de atracción hacia la configuración deseada
        verbose : si True, imprime progreso
    Retorna:
        q_sol   : vector articular alcanzado
        success : True si convergió, False si excedió max_iter
        history : lista con trayectorias articulares
    """

    # Asegurar que Tdes es SE3
    if not isinstance(Tdes, sm.SE3):
        Tdes = sm.SE3(Tdes)

    q = np.array(q0, dtype=float)
    history = [q.copy()]

    for k in range(max_iter):
        # Pose actual
        Tcur = robot.fkine(q)
        
        # Error de posición
        dP = Tdes.t - Tcur.t
        
        # Error de orientación (vector de ángulo-eje aproximado)
        Rerr = Tcur.R.T @ Tdes.R

        eR = sm.base.trlog(Rerr, twist=True)  # parte angular del twist

        # Vector de error 6D
        e = np.hstack((dP, eR))
        norm_e = np.linalg.norm(e)
        
        if norm_e < tol:
            if verbose:
                print(f"Convergencia en {k} iteraciones, error = {norm_e:.6f}")
            return q, True, history
        
        # Jacobiana completa 6×n
        J = robot.jacob0(q)
        
        # Damped least squares
        Jt = J.T
        dq = Jt @ np.linalg.inv(J @ Jt + lam**2 * np.eye(6)) @ (e * v_d * Ts)
        
        # Atracción hacia la configuración deseada
        if conf is not None:
            bias = np.zeros_like(q)
            bias[0] = -alpha * conf[0]
            bias[2] = -alpha * conf[1] # codo 
            bias[4] = -alpha * conf[2] # muñeca 
            dq += bias
        
        # Actualizar articulaciones
        q = q + dq
        history.append(q.copy())
        
        if verbose and k % 50 == 0:
            print(f"Iter {k}: error = {norm_e:.6f}")

    if verbose:
        print(f"No convergió tras {max_iter} iteraciones, error final = {norm_e:.6f}")
    return q, False, history

robot = irb140_clase()

qz =np.zeros(6)
n_acierto = 0
n_iter = 10
for i in range(0,n_iter):
    q_deseado = (np.random.rand(6)-0.5)*2*np.pi
    POSE = robot.fkine(q_deseado)
    conf = robot.get_config(q_deseado)
    q, success, history = ik_iterative(robot, POSE, qz , conf, max_iter=5000)
    if np.linalg.norm(q_deseado-q):
        n_acierto += 1
        print(f"ERROR:\nDeseado: {q_deseado}    config={conf}")
        print("Alcanzado: ",q)
    else:
        print(f"ERROR:\nDeseado: {q_deseado}    config={conf}")
        print("Alcanzado: ",q)

print("Cantidad de corridas: ",n_iter,"\nCantidad de aciertos: ",n_acierto)



ERROR:
Deseado: [ 1.40999243  1.02544329  3.04702752  2.40217975 -0.67591436  2.70774012]    config=[ 1.  1. -1.]
Alcanzado:  [ 1.28218698 -0.72304525 -0.08353886  0.49397858 -0.91810235 -1.38073098]
ERROR:
Deseado: [-0.34392346 -2.60876134  0.25603787 -0.4914035  -2.8250401   1.41580163]    config=[-1. -1. -1.]
Alcanzado:  [ 2.95663102  0.47405636  0.92061847  3.0871993  14.71292962 -1.30117855]
ERROR:
Deseado: [-1.99575075 -3.06726958 -1.59337477 -2.83639506 -2.84552881 -1.35596514]    config=[ 1.  1. -1.]
Alcanzado:  [-2.88811271  3.08566671 -1.51671627  3.46847046 -2.62880196  5.59283754]
ERROR:
Deseado: [-1.50719195 -0.4712596  -1.00685597 -0.50789043  1.65011554  2.51505986]    config=[-1. -1.  1.]
Alcanzado:  [ 2.05490846 -0.39973563 -0.65028914 -2.19259967 -1.32198735 -0.59426978]
ERROR:
Deseado: [ 2.49710686  0.57196439  2.05104598 -0.17816445 -0.46433983  2.22421917]    config=[-1.  1. -1.]
Alcanzado:  [-0.55845934 -1.706263    1.07471845 53.11792119  1.42843588  1.18156754]


In [30]:
q

array([  -2.34595943,    1.09172846,   -2.95199043, -152.32478345,
         18.01138509,  140.86430096])